In [ ]:
import os
import sys
import subprocess
import shutil
import pandas as pd
import numpy as np
import xmltodict
import json
import hashlib
import requests
import getpass
from requests import Response
from time import sleep

import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import table

HOMEDIR="."

In [ ]:
sys.path.append("../../lib")
print(sys.path)
from pyutils import *
from pdutils import *
from pdpltutils import *
from gputils import *
from iputils import *
from DIGIHandler import DIGIHandler

### Using the DIGIHandler class

In [ ]:
dh = DIGIHandler()
dh.setPassword()
dh.getModemStatus(output=False)

In [ ]:
dh.toggleSIM()

In [ ]:
dh.waitForConnect()

In [ ]:
dh.pingTest(dest="cloudlet020.elijah.cs.cmu.edu",c=10)

In [ ]:
pingres =  dh.pingTest(dest="cloudlet020.elijah.cs.cmu.edu",c=2)
print(pingres)

In [ ]:
linelib = []
''' Alternately run a ping test on each SIM '''
for ii in range(0,5):
    dh.waitForConnect()
    sleep(5)
    sno, sname = dh.getCurrentSIM()
    print(sno,sname)
    res = dh.pingTest(dest="cloudlet020.elijah.cs.cmu.edu",c=10)
    linelib.extend([f"{humandatenow()},{sno},{sname},{line}" for line in res['stdout']])
    dh.toggleSIM()

print(linelib)

In [ ]:
''' Process the ping output into a dataframe '''
linelib2 = [line.split(",") for line in linelib]

collst = ['TIMESTAMP','SIMNO','SIMNAME','RAWLINE']
tdfx = pd.DataFrame([line for line in linelib2 if len(line) == len(collst)], 
                    columns=['TIMESTAMP','SIMNO','SIMNAME','RAWLINE'])
tdfx = to_ts(tdfx,format = '%Y-%m-%d-%H-%M-%S-%f')
tdfx['HEADERLINE'] = tdfx.RAWLINE.str.startswith("PING")
tdfx['PINGLINE'] = tdfx.RAWLINE.str.contains("time=")
tdfx['STATLINE'] = tdfx.RAWLINE.str.startswith("rtt")
tdfx = tdfx[tdfx.HEADERLINE | tdfx.PINGLINE | tdfx.STATLINE]
# dumpdf(tdfx,head=len(tdfx))

import re
def parsePing(row):
    time = np.nan
    dest = None
    src = None
    statlst = None
    if row.PINGLINE:
        rec = re.compile("time=(.*) ")
        time = float(rec.findall(row.RAWLINE)[0])
    if row.HEADERLINE:
        rec = re.compile("PING (\S*) ")
        dest = rec.findall(row.RAWLINE)[0]
        rec = re.compile(" (\S*)\: ")
        src = rec.findall(row.RAWLINE)[0]
    if row.STATLINE:
        statlst = [float(stat) for stat in row.RAWLINE.replace(" ms","").replace("mdev = ","").split("/")[-4:]]
        print(statlst)
#     print(time,dest,src,statlst)
    return time,dest,src,statlst
tdfx[['TIME','DEST','SRC','STATLST']] = tdfx.apply(parsePing,axis=1,result_type="expand")
# tdfx.apply(parsePing,axis=1,result_type="expand")
# dumpdf(tdfx,head=len(tdfx))

tdfx['STATLST'] = tdfx.sort_index(ascending=False).STATLST.ffill().sort_index()

tdfx['DEST'] = tdfx.DEST.ffill()

tdfx['SRC'] = tdfx.SRC.ffill()

tdfy = pd.DataFrame(list(tdfx.STATLST),columns=['min','avg','max','mdev'])
dumpdf(tdfy)

tdfx = pd.concat([tdfx.reset_index(drop=True),tdfy],axis=1)
dumpdf(tdfx)

collst = ['TIMESTAMP', 'SIMNO', 'SIMNAME', 'TIME', 'DEST', 'SRC', 'min','avg','max','mdev']
tdfy = tdfx[tdfx.PINGLINE][collst].copy()
dumpdf(tdfy)

In [ ]:
writejoin(tdfy.set_index('TIMESTAMP'),".","pingtest_tmp.csv")

## Remote Manager Web API

In [ ]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="remotemanager.digi.com"            
api="https://%s/ws/v1/devices/inventory/00000000-00000000-0004F3FF-FF94F583/" % host

# user = input("Username for %s: " % host)
user="USER"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="<PW>"

# Retaining a session when doing multiple requests can speed up your requests
session = requests.Session()
session.auth = (user, password)

rsp = session.get(api)
rsp.raise_for_status()
print("Status code:", rsp.status_code)
# print("Result text:", rsp.text)

stts=json.loads(rsp.text)

for xx in stts.keys():
    print(f"{xx} -> {stts[xx]}")

In [ ]:

KEYLST = ['connection_status','provider','apn','carrier','last_connect','last_disconnect']
def checkStatus(output=False,keylst=KEYLST):
    session = requests.Session()
    session.auth = (user, password)
    rsp = session.get(api)
    rsp.raise_for_status()
#     print(rsp.text)
    stts=json.loads(rsp.text)
    if (output):
        keyresult = " ".join([stts[key] for key in keylst if key in stts ])
        print(keyresult)
    del rsp
    del session
    return stts

TIMEOUTCT=20
TIMEOUTWT=2

def waitForConnect():
    for ii in range(0,TIMEOUTCT):
        stts = checkStatus()
        connectstat=stts['connection_status']
        carrier = stts['provider']
        if (connectstat=='connected'):
            print(f"Connected to {carrier} last_disconnect={stts['last_disconnect']}")
#             break
        print(f"{ii}")
        sleep(TIMEOUTWT)
                  
# waitForConnect()

In [ ]:
checkStatus(output=True)

## Local Web API

In [ ]:
#!python3
# Python example using requests library 
# Use 'pip install requests' to install the requests library.           

host="192.168.2.1"            
api="https://%s/cgi-bin/config.cgi/value/service/ssh" % host
print(api)
# https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh

# user = input("Username for %s: " % host)
user="admin"
# password = getpass.getpass("Password for user %s at %s:" % (user, host))
password="<PW>"

# # Retaining a session when doing multiple requests can speed up your requests
# session = requests.Session()
# session.auth = (user, password)

# rsp = session.get(api)
# rsp.raise_for_status()
# print("Status code:", rsp.status_code)
# # print("Result text:", rsp.text)

cmd=f"curl -k -u {user}:{password} {api}"
print(cmd)
response = cmd_all(cmd, output=True)

# stts=json.loads(rsp.text)

# for xx in stts.keys():
#     print(f"{xx} -> {stts[xx]}")

In [ ]:
# config network modem wwan1 sim_slot
api="https://%s/cgi-bin/config.cgi/value/network/interface/wwan1" % host
cmd=f"curl -k -u {user}:{password} {api} -X GET"
print(cmd)
response = cmd_all(cmd, output=True)

In [ ]:
cmd="curl -k -u admin:<PW> https://192.168.2.1/cgi-bin/config.cgi/value/service/ssh"
cmd_all(cmd,output=True)